In [ ]:
import json
import os
from pathlib import Path
from typing import Dict, List

class teamEvaluator:
    """
    Create json with 3 lists
    - last calculated week
    - all season
    - last 3 games
    - last season - hardcoded
    """
    def __init__(self, current_week):

        with open('team_evaluator.json', 'r', encoding='utf-8') as f:
            self.evaluator_json  = json.load(f)
        
        if not self.evaluator_json:
            print(f"⚠️  Evaluator file not found: team_evaluator.json")
            return None

        self.last_loaded_week = self.evaluator_json["last_loaded_week"]
        self.current_week = current_week

        if not self.last_loaded_week: self.last_loaded_week  = 0

    def _update_stats(self, lower_week: int, section_name: str) -> Dict[str, Dict[str, int]]:

        team_stats = self.evaluator_json[section_name]
        
        periods_to_update = range(lower_week, self.current_week)
        calendar_path = Path('./calendar')

        for period in periods_to_update:

            # Get all week JSON files
            week_file = sorted(calendar_path.glob(f'week_{period}.json'))[0]

            if not week_file:
                print(f"No week {period} files found in {calendar_path}")
                return team_stats

            # Process each week file
        
            with open(week_file, 'r', encoding='utf-8') as f:
                matches = json.load(f)
            
            # Process each match
            for match in matches:
                    # Only process finished matches (matchState 7)
                    if match.get('matchState') == 7:
                        local_score = match.get('localScore')
                        visitor_score = match.get('visitorScore')
                        
                        # Skip if scores are null
                        if local_score is None or visitor_score is None:
                            continue
                        
                        local_team = match['local']['name']
                        visitor_team = match['visitor']['name']
                        
                        # Initialize team stats if not exists
                        if local_team not in team_stats:
                            team_stats[local_team] = {
                                'gamesPlayed': 0,
                                'goalsScoredLocal': 0,
                                'goalsAgainstLocal': 0,
                                'goalsScoredVisitor': 0,
                                'goalsAgainstVisitor': 0,
                                'wonLocal': 0,
                                'wonVisitor':0,
                                'lostLocal': 0,
                                'lostVisitor': 0,
                                'drawLocal': 0,
                                'drawVisitor': 0,
                                'teamId': match['local']['id'],
                                'shortName': match['local']['shortName']
                            }
                        
                        if visitor_team not in team_stats:
                            team_stats[visitor_team] = {
                                'gamesPlayed': 0,
                                'goalsScoredLocal': 0,
                                'goalsAgainstLocal': 0,
                                'goalsScoredVisitor': 0,
                                'goalsAgainstVisitor': 0,
                                'wonLocal': 0,
                                'wonVisitor':0,
                                'lostLocal': 0,
                                'lostVisitor': 0,
                                'drawLocal': 0,
                                'drawVisitor': 0,
                                'teamId': match['visitor']['id'],
                                'shortName': match['visitor']['shortName']
                            }
                        
                        # Update stats
                        team_stats[local_team]['goalsScoredLocal'] += local_score
                        team_stats[local_team]['goalsAgainstLocal'] += visitor_score
                        team_stats[visitor_team]['goalsScoredVisitor'] += visitor_score
                        team_stats[visitor_team]['goalsAgainstVisitor'] += local_score
                        team_stats[local_team]['gamesPlayed'] += 1
                        team_stats[visitor_team]['gamesPlayed'] += 1

                        if local_score > visitor_score:
                            team_stats[visitor_team]['wonLocal'] += 1
                            team_stats[local_team]['lostVisitor'] += 1
                        elif local_score < visitor_score:
                            team_stats[visitor_team]['wonVisitor'] += 1
                            team_stats[local_team]['lostLocal'] += 1
                        else:
                            team_stats[visitor_team]['drawVisitor'] += 1
                            team_stats[local_team]['drawLocal'] += 1


        return team_stats

    def update(self):
        self._update_stats(self.last_loaded_week, "season_overall")
        self._update_stats(self.current_week-3, "L3_games")

if __name__ == '__main__':
    # Calculate team strength
    evaluator = teamEvaluator(12)
    evaluator.update()

{'Villarreal CF': {'goalsScoredLocal': 12, 'goalsAgainstLocal': 3, 'goalsScoredVisitor': 6, 'goalsAgainstVisitor': 7, 'wonLocal': 2, 'wonVisitor': 2, 'lostLocal': 0, 'lostVisitor': 4, 'drawLocal': 1, 'drawVisitor': 1, 'teamId': '20', 'shortName': 'VIL'}, 'Real Oviedo': {'goalsScoredLocal': 2, 'goalsAgainstLocal': 10, 'goalsScoredVisitor': 5, 'goalsAgainstVisitor': 9, 'wonLocal': 3, 'wonVisitor': 1, 'lostLocal': 4, 'lostVisitor': 1, 'drawLocal': 0, 'drawVisitor': 1, 'teamId': '157', 'shortName': 'OVI'}, 'Valencia CF': {'goalsScoredLocal': 7, 'goalsAgainstLocal': 5, 'goalsScoredVisitor': 3, 'goalsAgainstVisitor': 11, 'wonLocal': 3, 'wonVisitor': 0, 'lostLocal': 2, 'lostVisitor': 2, 'drawLocal': 1, 'drawVisitor': 2, 'teamId': '18', 'shortName': 'VAL'}, 'Real Sociedad': {'goalsScoredLocal': 6, 'goalsAgainstLocal': 6, 'goalsScoredVisitor': 4, 'goalsAgainstVisitor': 8, 'wonLocal': 3, 'wonVisitor': 0, 'lostLocal': 2, 'lostVisitor': 2, 'drawLocal': 1, 'drawVisitor': 2, 'teamId': '16', 'shortNa